In [19]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.


#    example which maximizes the sum of a list of integers
#    each of which can be 0 or 1

import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_resistance", random.randint, 1000, 100000)

# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
# toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_resistance, 2) # Voltage divider
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_resistance, 4) # Parallel resistance

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ('fitness') function to be maximized
def evalVoltDivider(individual):
    if individual[0] > 0 and individual[1] > 0:
        return (10 * individual[1] / (individual[0] + individual[1])),  # 10V voltage divider
    else:
        return 0,

def evalParallelRes(individual):
    if individual[0] > 0 and individual[1] > 0 and individual[2] > 0 and individual[3] > 0:
        return (1 / (individual[0]**-1 + individual[1]**-1 + individual[2]**-1 + individual[3]**-1)),  # 4 Parallel resistors
    else:
        return 0,

#----------
# Operator registration
#----------
# register the goal / fitness function
# toolbox.register("evaluate", evalVoltDivider) # Optimizes the voltage divider problem
toolbox.register("evaluate", evalParallelRes) # Optimizes the parallel resistance problem

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selTournament, tournsize=3)

In [20]:
def main():
    random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=300)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.1

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0

    # Begin the evolution
    # while max(fits) < 9.87 and g < 3000: # Voltage divider
    while max(fits) < 24500:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

In [21]:
if __name__ == "__main__":
    main()

Start of evolution
  Evaluated 300 individuals
-- Generation 1 --
  Evaluated 199 individuals
  Min 0.0
  Max 22279.022514711905
  Avg 12381.757606659776
  Std 4968.0410711215345
-- Generation 2 --
  Evaluated 191 individuals
  Min 0.0
  Max 22708.644324296423
  Avg 15511.07414479483
  Std 4849.247681515259
-- Generation 3 --
  Evaluated 195 individuals
  Min 0.0
  Max 23350.494223181893
  Avg 17972.545094751316
  Std 4589.151156042213
-- Generation 4 --
  Evaluated 195 individuals
  Min 0.0
  Max 23518.201726095707
  Avg 19579.92225168918
  Std 4508.441051621855
-- Generation 5 --
  Evaluated 209 individuals
  Min 0.0
  Max 23841.92003773056
  Avg 20113.569271562224
  Std 5460.981386204039
-- Generation 6 --
  Evaluated 189 individuals
  Min 0.0
  Max 23858.036932054834
  Avg 20857.345971436553
  Std 5767.337604621637
-- Generation 7 --
  Evaluated 216 individuals
  Min 0.0
  Max 24121.041492215667
  Avg 21642.14922827481
  Std 5329.59196973659
-- Generation 8 --
  Evaluated 204 indiv

KeyboardInterrupt: 